In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from ripser import ripser
from persim import plot_diagrams
from scipy.spatial.distance import pdist, squareform
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.svm import LinearSVC, SVC, SVR
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix
from gtda.time_series import TakensEmbedding
from PyEMD import EMD
from statsmodels.tsa.stattools import adfuller
from pylab import mpl
from sklearn import preprocessing

%matplotlib qt

In [ ]:
# define load data func
def load_data_from_csv(filepath: str) -> np.ndarray:
    csv_file = pd.read_csv(filepath)
    school_num = csv_file.shape[0]
    year_num = csv_file.shape[1] - 2    # except first and last column
    data_dim = len(csv_file.iloc[0, 1].strip().split())
    school_names = tuple(csv_file["SchoolName"])
    data = np.zeros((school_num, year_num, data_dim))
    for i, _ in enumerate(school_names):
        data_row = list(csv_file.iloc[i, 1:year_num+1])
        data_row = [_.strip().split() for _ in data_row]
        data_row = [float(_) for l in data_row for _ in l]
        data_row = np.array(data_row).reshape((year_num, -1))
        data[i, :, :] = data_row
    return data, school_names

In [ ]:
# load data
data, school_names = load_data_from_csv("../dataset/all-data.csv")
school_n, year_n, data_dim = data.shape
idx2sch = {k:v for k,v in enumerate(school_names)}

In [ ]:
# select the data we use (see README.md)
selected_data_idx = np.array([0, 2, 7, 11, 15, 19])
selected_data = data[:, :, selected_data_idx]
selected_data = selected_data.transpose(0, 2, 1)
print(selected_data.shape)

In [ ]:
# more data!
more_data_path = "../dataset/more-school-all.npy"
more_data = np.load(more_data_path)
selected_data = np.concatenate((selected_data, more_data), axis=0)
print(selected_data.shape)

In [ ]:
# more data!
more_data_path = "../dataset/top20.npy"
more_data = np.load(more_data_path)
selected_data = np.zeros((20,6,12))
selected_data[:, 0:2, :] = more_data[:, 0:2, :]
# print(selected_data[0,:,:])
selected_data[:, 2, :] = more_data[:, 2, :] + more_data[:, 3, :]
selected_data[:, 3:6, :] = more_data[:, 4:7, :]
print(selected_data.shape)
print(selected_data[0,:,:])

In [ ]:
selected_data_scaled = preprocessing.scale(selected_data.reshape(-1, 10), axis=1)

D_state_list = []
G_state_list = []

for i in range(selected_data_scaled.shape[0]):
    if selected_data_scaled[i][-2] == 0:
        G_state_list.append(i)
    else:
        if selected_data_scaled[i][-1]/selected_data_scaled[i][-2] > 1:
            G_state_list.append(i)
        else:
            D_state_list.append(i)

print(G_state_list)
print(D_state_list)

pca = PCA(n_components=2)
x_2d = pca.fit_transform(selected_data_scaled)
g = plt.scatter(x_2d[G_state_list, 0], x_2d[G_state_list, 1], s=65, color='red', label="Samples with growing trend")
d = plt.scatter(x_2d[D_state_list, 0], x_2d[D_state_list, 1], s=65, color='royalblue', label="Samples with decreasing trend")

ax = plt.gca()
ax.axes.xaxis.set_ticklabels([])
ax.axes.yaxis.set_ticklabels([])
ax.spines[:].set_linewidth('2.0')

plt.legend(fontsize=18)
plt.title("Samples distribution in original space", fontsize=24, pad=20)

plt.show()

In [ ]:
# define trend-generation func
def generate_trend_matrix(statis_matrix: np.array, dot_prsv=2) -> np.array:
    assert len(statis_matrix.shape) == 2
    m, n = statis_matrix.shape
    trend_matrix = np.empty((m, n-1))
    for row in range(0, m):
        for col in range(0, n-1):
            if statis_matrix[row, col] != 0:
                trend_matrix[row, col] = round(statis_matrix[row, col+1] / statis_matrix[row, col], dot_prsv)
            else:
                trend_matrix[row, col] = 10
    return trend_matrix

In [ ]:
enhanced_data = []
reshaped = selected_data.reshape(-1, 10)
# reshaped = [generate_trend_matrix(reshaped[i, :].reshape(1,-1)).squeeze() for i in range(reshaped.shape[0])]
# reshaped = np.array(reshaped)
for i in range(reshaped.shape[0]):
    d = reshaped[i, :]
    d = (d-np.mean(d)) / np.std(d)
    # ATTENTION
    enhanced_data.append([[d[j], d[j+1], d[j+2]] for j in range(len(d)-3)])
    # enhanced_data.append([[d[j], d[j+1], d[j+2]] for j in range(len(d)-2)])
enhanced_data = np.array(enhanced_data)
print(enhanced_data.shape)
print(enhanced_data[0,:,:])

In [ ]:
mpl.rcParams['font.size'] = 18
mpl.rcParams['font.family'] = "Microsoft Reference Sans Serif"
mpl.rcParams['legend.fontsize'] = 18
i = 0
r = ripser(enhanced_data[i,:,:])
plt.subplot(2,3,1)
plot_diagrams(r['dgms'], diagonal=False, size=40)

mpl.rcParams['font.size'] = 18
mpl.rcParams['font.family'] = "Microsoft Reference Sans Serif"
i = 3
r = ripser(enhanced_data[i,:,:])
plt.subplot(2,3,2)
plot_diagrams(r['dgms'], diagonal=False, size=40)

mpl.rcParams['font.size'] = 18
mpl.rcParams['font.family'] = "Microsoft Reference Sans Serif"
i = 2
r = ripser(enhanced_data[i,:,:])
plt.subplot(2,3,3)
plot_diagrams(r['dgms'], diagonal=False, size=40)

mpl.rcParams['font.size'] = 18
mpl.rcParams['font.family'] = "Microsoft Reference Sans Serif"
i = 4
r = ripser(enhanced_data[i,:,:])
plt.subplot(2,3,4)
plot_diagrams(r['dgms'], diagonal=False, size=40)

mpl.rcParams['font.size'] = 18
mpl.rcParams['font.family'] = "Microsoft Reference Sans Serif"
i = 5
r = ripser(enhanced_data[i,:,:])
plt.subplot(2,3,5)
plot_diagrams(r['dgms'], diagonal=False, size=40)

mpl.rcParams['font.size'] = 18
mpl.rcParams['font.family'] = "Microsoft Reference Sans Serif"
i = 10
r = ripser(enhanced_data[i,:,:])
plt.subplot(2,3,6)
plot_diagrams(r['dgms'], diagonal=False, size=40)

In [ ]:
# for i in range(2):
# for i in range(enhanced_data.shape[0]):
feature_list = []
for i in range(enhanced_data.shape[0]):
    r = ripser(enhanced_data[i, :, :])
    feature = np.delete(r['dgms'][0], -1, axis=0)[:, 1]
    # feature_list.append(np.array([feature.shape[0], np.sum(feature), np.mean(feature), np.max(feature), np.min(feature), len([_ for _ in feature if _>0.5*np.max(feature)]), len([_ for _ in feature if _>np.mean(feature)])]))
    f = [feature.shape[0], np.sum(feature), np.mean(feature), np.std(feature), np.max(feature), np.min(feature), len([_ for _ in feature if _>0.5*np.max(feature)]), len([_ for _ in feature if _>np.mean(feature)])]
    f = [np.round(_, 2) for _ in f]
    feature_list.append(np.array(f))

In [ ]:
# get trends
trends = np.array([generate_trend_matrix(reshaped[i, :].reshape(1,-1)).squeeze() for i in range(reshaped.shape[0])])
trends = trends[:, -1].reshape(-1, 1)
# print(trends)
feature_pd = pd.DataFrame(np.concatenate((np.array(feature_list), trends), axis=1), \
    columns=['Number of TDA barcode points', 'Sum of lifetime', 'Average of lifetime', 'Std of lifetime', 'Max of lifetime', \
        'Min of lifetime', 'Number of points bigger than 0.5*max', 'Number of points bigger than average', 'Trend'])
# print(feature_pd)
# feature_pd.to_csv('./feature_map.csv')

In [ ]:
# Sum, Max and Min of Lifetime are selected
print(feature_pd.corr())

In [ ]:
# See classification bounds
feature_np = np.array(feature_list)
print(feature_np.shape)
# data_1d = feature_np.reshape(1, -1).squeeze()
# print(data_1d.shape)

# data_1d = sorted(data_1d, reverse=False)

# print(data_1d.index(1.0))
# print(data_1d[0], data_1d[800], data_1d[1600])
# print(data_1d[0], data_1d[600], data_1d[1200], data_1d[1800])

D_state_list = []
G_state_list = []

for i in range(selected_data_scaled.shape[0]):
    if selected_data_scaled[i][-2] == 0:
        G_state_list.append(i)
    else:
        if selected_data_scaled[i][-1]/selected_data_scaled[i][-2] > 1:
            G_state_list.append(i)
        else:
            D_state_list.append(i)

pca = PCA(n_components=2)
feature_np_2d = pca.fit_transform(feature_np)
g = plt.scatter(feature_np_2d[G_state_list, 0], feature_np_2d[G_state_list, 1], s=65, color='red', label="Samples with growing trend")
d = plt.scatter(feature_np_2d[D_state_list, 0], feature_np_2d[D_state_list, 1], s=65, color='royalblue', label="Samples with decreasing trend")

ax = plt.gca()
ax.axes.xaxis.set_ticklabels([])
ax.axes.yaxis.set_ticklabels([])
ax.spines[:].set_linewidth('2.0')

plt.legend(fontsize=18)
plt.title("Samples distribution in Persistence Diagrams", fontsize=24, pad=20)

plt.show()


In [ ]:
# X = feature_pd[['Sum of lifetime', 'Max of lifetime', 'Min of lifetime']]
X = feature_pd[['Number of TDA barcode points', 'Sum of lifetime', 'Average of lifetime', 'Std of lifetime', 'Max of lifetime', \
        'Min of lifetime', 'Number of points bigger than 0.5*max', 'Number of points bigger than average']]
Y = feature_pd[['Trend']]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

In [ ]:
# 0: trend <= 1
# 1: 1 < trend <= 1.2
# 2: 1.2 < trend <= 1.5
# 3: 1.5 < trend
# def P_idx(x):
#     if x<=1:
#         return 0
#     elif x>1 and x<=1.25:
#         return 1
#     elif x>1.25 and x<=1.5:
#         return 2
#     else:
#         return 3

# def P_idx(x):
#     if x<=1:
#         return 0
#     elif x>1 and x<=1.5:
#         return 1
#     else:
#         return 2

def P_idx(x):
    if x<=1:
        return 0
    else:
        return 1

In [ ]:
# import numpy as np
# from sklearn import svm

# def gaussianKernelGramMatrixFull(X1, X2, sigma=0.1):
#     """(Pre)calculates Gram Matrix K"""

#     gram_matrix = np.zeros((X1.shape[0], X2.shape[0]))
#     for i, x1 in enumerate(X1):
#         for j, x2 in enumerate(X2):
#             x1 = x1.flatten()
#             x2 = x2.flatten()
#             gram_matrix[i, j] = np.exp(- np.sum( np.power((x1 - x2),2) ) / float( 2*(sigma**2) ) )
#     return gram_matrix

# X=...
# y=...
# Xval=...

# C=0.1
# clf = svm.SVC(C = C, kernel="precomputed")
# model = clf.fit( gaussianKernelGramMatrixFull(X,X), y )

# p = model.predict( gaussianKernelGramMatrixFull(Xval, X) )
def gaussianKernelGramMatrixFull(X1, X2, sigma=0.1):
    """(Pre)calculates Gram Matrix K"""

    gram_matrix = np.zeros((X1.shape[0], X2.shape[0]))
    for i, x1 in enumerate(X1):
        for j, x2 in enumerate(X2):
            x1 = x1.flatten()
            x2 = x2.flatten()
            gram_matrix[i, j] = np.exp(- np.sum( np.power((x1 - x2),2) ) / float( 2*(sigma**2) ) )
    return gram_matrix

In [ ]:
# print(list(Y_train['Trend']))
# print(list(Y_test))
Y_train = Y_train.applymap(P_idx)
Y_test = Y_test.applymap(P_idx)

C = 0.1
clf = SVC(C=C, kernel='linear')
clf.fit(X_train, np.ravel(Y_train))

print(precision_score(y_true=Y_test, y_pred=clf.predict(X_test), average='macro'))
print(recall_score(y_true=Y_test, y_pred=clf.predict(X_test), average='macro'))
print(accuracy_score(y_true=Y_test, y_pred=clf.predict(X_test)))
print(f1_score(y_true=Y_test, y_pred=clf.predict(X_test), average='macro'))

In [ ]:
cm = confusion_matrix(y_true=Y_test, y_pred=clf.predict(X_test))

# 某类的FP：该列所有元素之和减去该列的TP.
# 某类的FN：该行所有元素之和减去该行的TP.
# 某类的TN：整个混淆矩阵之和减去该类的（TP+FP+FN）
print(cm)
precision = 7 / 12  # TP / TP+FP
recall = 7 / 11     # TP / TP+FN
accuracy = 42 / 75   # TP+TN / ALL
f1 = 2*(precision*recall)/(precision+recall)

print(precision, recall, accuracy, f1)

In [ ]:
# Linear Regression Predict, do not use :)
X = feature_pd[['Sum of lifetime', 'Max of lifetime', 'Min of lifetime']]
Y = feature_pd[['Trend']]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=1)
linreg = LinearRegression()
model = linreg.fit(X_train, Y_train)
print(linreg.intercept_)
print(linreg.coef_)
Y_pred = linreg.predict(X_test)
Y_test = np.array(Y_test)

# accu = 0
# for i in range(Y_pred.shape[0]):
#     if P_idx(Y_pred[i]) == P_idx(Y_test[i]):
#         accu += 1

# print(accu / Y_pred.shape[0])

# print(Y_pred)
# print(Y_test)
print(Y_pred - Y_test)

In [ ]:
# X = feature_pd[['Sum of lifetime', 'Max of lifetime', 'Min of lifetime']]
X = feature_pd[['Number of TDA barcode points', 'Sum of lifetime', 'Average of lifetime', 'Std of lifetime', 'Max of lifetime', \
        'Min of lifetime', 'Number of points bigger than 0.5*max', 'Number of points bigger than average']]
Y = feature_pd[['Trend']]

# Y = Y.applymap(P_idx)

# X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

X = X.to_numpy()
Y = Y.to_numpy()

pred = []
for i in range(120):
    X_test = X[i,1:]
    X_train = np.delete(X, -1, axis=1)
    X_train = np.delete(X_train, i, axis=0)
    Y_train = np.delete(Y, i, axis=0)
    svr = SVR(C=1.0, epsilon=0.2, kernel='linear')
    svr.fit(X_train, np.ravel(Y_train))
    # print(X_test)
    pred.append(float(svr.predict(X_test.reshape(1,-1))))

# for i in range(120):
#     X_test = X[i,1:]
#     X_train = np.delete(X, -1, axis=1)
#     X_train = np.delete(X_train, i, axis=0)
#     Y_train = np.delete(Y, i, axis=0)
    
#     C = 0.1
#     clf = SVC(C=C, kernel='linear')
#     clf.fit(X_train, np.ravel(Y_train))
#     pred.append(int(clf.predict(X_test.reshape(1,-1))))
print(pred[:120])

In [ ]:
print(reshaped.shape)

In [ ]:
X = np.arange(8).reshape(2, 4)
TE = TakensEmbedding(time_delay=1, dimension=3)
print(X)
print(TE.fit_transform(X))
print(TE.fit_transform(X).shape)

In [ ]:
TE = TakensEmbedding(time_delay=1, dimension=3)
enhanced_data = TE.fit_transform(reshaped[:, :-1])
print(enhanced_data.shape)

In [ ]:
print(enhanced_data[0,:,:])